<a href="https://colab.research.google.com/github/namtoptall/DataScience/blob/main/iception_milestone_project_data_science.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Huấn luyện mô hình : Model training
**Lưu ý ⚠ em huấn luyện mô hình trên google colab nên nếu muốn chạy lại toàn bộ thì cần set up những tài khoản sau :**
> tài khoản kaggle và kaggle API cho việc download bộ dữ liệu

> tài khoản google Drive để kết nối với kho dữ liệu cá nhân



### kết nối với google drive và kaggle

Với kaggle : thực hiện tải tệp kaggle.json(Chứa thông tin tài khoản) lên

In [ ]:
from google.colab import files
files.upload()  # Thực hiện tải tệp kaggle.json lên từ máy tính (phải có tài khoản kaggle nha)

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"namdz242221","key":"de869a9575f6ac51974bca65557d88a7"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Mount google drive content vào trong notebook

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!kaggle datasets download -d namdz242221/dsp305x-01-a-vn-dataset

Dataset URL: https://www.kaggle.com/datasets/namdz242221/dsp305x-01-a-vn-dataset
License(s): MIT
 98% 518M/530M [00:14<00:00, 46.4MB/s]
100% 530M/530M [00:14<00:00, 39.1MB/s]


unzip the dataset

In [ ]:
!unzip dsp305x-01-a-vn-dataset.zip -d /content/dataset


Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
  inflating: /content/dataset/DSP305x_01-A_VN-dataset/val/Peach blossom/000629.jpg  
  inflating: /content/dataset/DSP305x_01-A_VN-dataset/val/Peach blossom/000630.jpg  
  inflating: /content/dataset/DSP305x_01-A_VN-dataset/val/Peach blossom/000631.jpg  
  inflating: /content/dataset/DSP305x_01-A_VN-dataset/val/Peach blossom/000632.jpg  
  inflating: /content/dataset/DSP305x_01-A_VN-dataset/val/Peach blossom/000633.jpg  
  inflating: /content/dataset/DSP305x_01-A_VN-dataset/val/Peach blossom/000637.jpg  
  inflating: /content/dataset/DSP305x_01-A_VN-dataset/val/Peach blossom/000638.jpg  
  inflating: /content/dataset/DSP305x_01-A_VN-dataset/val/Peach blossom/000639.jpg  
  inflating: /content/dataset/DSP305x_01-A_VN-dataset/val/Peach blossom/000641.jpg  
  inflating: /content/dataset/DSP305x_01-A_VN-dataset/val/Peach blossom/000642.jpg  
  inflating: /content/dataset/DSP305x_01-A_VN-dataset/val/Peach blossom/000643.jpg  
  inflat

In [ ]:
!ls /content/dataset

DSP305x_01-A_VN-dataset


check content trong tệp

In [ ]:
!ls /content/dataset

DSP305x_01-A_VN-dataset


In [ ]:
import os
dataset_dir = "/content/dataset"
for root, dirs, files in os.walk(dataset_dir):
    print(root, "contains", len(files), "files.")


/content/dataset contains 0 files.
/content/dataset/DSP305x_01-A_VN-dataset contains 1 files.
/content/dataset/DSP305x_01-A_VN-dataset/train contains 0 files.
/content/dataset/DSP305x_01-A_VN-dataset/train/Plumeria contains 1995 files.
/content/dataset/DSP305x_01-A_VN-dataset/train/Peach blossom contains 2002 files.
/content/dataset/DSP305x_01-A_VN-dataset/train/Orchid contains 2041 files.
/content/dataset/DSP305x_01-A_VN-dataset/train/Rose contains 2061 files.
/content/dataset/DSP305x_01-A_VN-dataset/train/Lotus contains 2035 files.
/content/dataset/DSP305x_01-A_VN-dataset/train/Bougainvillea contains 2156 files.
/content/dataset/DSP305x_01-A_VN-dataset/train/Peony contains 2038 files.
/content/dataset/DSP305x_01-A_VN-dataset/train/Sunflower contains 1997 files.
/content/dataset/DSP305x_01-A_VN-dataset/train/Chrysanthemum contains 2002 files.
/content/dataset/DSP305x_01-A_VN-dataset/train/Plum blossom contains 2044 files.
/content/dataset/DSP305x_01-A_VN-dataset/test contains 0 files.

check shape của từng ảnh

In [ ]:
from PIL import Image
import os
from collections import defaultdict

def check_unique_shapes(dataset_dir):
    # count the number of difference shapes
    shape_counts = defaultdict(int)

    for root, dirs, files in os.walk(dataset_dir):
        for file in files:
            if file.endswith(('.png', '.jpg', '.jpeg')):
                img_path = os.path.join(root, file)
                try:
                    with Image.open(img_path) as img:
                        shape = img.size  # Lấy kích thước (chiều rộng, chiều cao) của hình ảnh
                        shape_counts[shape] += 1
                except Exception as e:
                    print(f"Error processing image {img_path}: {e}")

    return shape_counts

# Ví dụ sử dụng
dataset_dir = "/content/dataset"
unique_shapes = check_unique_shapes(dataset_dir)

# In ra các shape và số lần xuất hiện
for shape, count in unique_shapes.items():
    print(f"Shape {shape} appears {count} times.")


Shape (224, 224) appears 36617 times.


👉 shape của những ảnh trong dataset là 224,224

In [ ]:
# xóa file metadata.json (chỉ có tác dụng khi upload data lên kaggle.)
!rm /content/dataset/dataset-metadata.json

rm: cannot remove '/content/dataset/dataset-metadata.json': No such file or directory


## một chút pre-check và labelling dataset

tải file .utils/ETL_functions.py từ thư mục đồ án

In [ ]:
from google.colab import files
files.upload()  # Thực hiện tải ETL_functions.py lên từ máy tính

Saving ETL_functions.py to ETL_functions.py


{'ETL_functions.py': b'import os\r\nimport numpy as np\r\nimport pandas as pd\r\nimport tensorflow as tf\r\nfrom PIL import Image\r\nfrom sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix\r\nimport itertools\r\nimport matplotlib.pyplot as plt\r\nimport datetime\r\n# set the max image pixels to None\r\nImage.MAX_IMAGE_PIXELS = None\r\nimport os\r\nfrom PIL import Image, UnidentifiedImageError\r\n\r\n# Constants\r\nTARGET_IMG_SIZE = 256\r\n\r\nCLASS_LABELS = [\r\n    "Lotus", \r\n    "Chrysanthemum",\r\n    "Peony",\r\n    "Orchid",\r\n    "Sunflower",\r\n    "Bougainvillea",\r\n    "Rose",\r\n    "peach blossom",\r\n    "Plumeria",\r\n    "Plum blossom"\r\n]\r\n\r\ndef extract_image_info(image_path: str) -> dict:\r\n    """\r\n    Extract information from an image file.\r\n\r\n    :param image_path: Full path to the image file.\r\n    :return: A dictionary with extracted image information or None if the file is not a valid image.\r\n    """\r\n    

In [ ]:
import ETL_functions as etl
from ETL_functions import *

## Mở bộ dữ liệu hoa

In [ ]:
train_path = "/content/dataset/DSP305x_01-A_VN-dataset/train"
test_path = "/content/dataset/DSP305x_01-A_VN-dataset/test"
val_path = "/content/dataset/DSP305x_01-A_VN-dataset/val"

In [ ]:
train= etl.load_dataset(train_path)
test = etl.load_dataset(test_path)
val = etl.load_dataset(val_path)
train

,ImgPath,FileType,Width,Height,Ratio,Mode,Bands,Transparency,Animated,Class
0,/content/dataset/DSP305x_01-A_VN-dataset/train...,jpg,224,224,1.0,RGB,R G B,False,False,Plumeria
1,/content/dataset/DSP305x_01-A_VN-dataset/train...,jpg,224,224,1.0,RGB,R G B,False,False,Plumeria
2,/content/dataset/DSP305x_01-A_VN-dataset/train...,jpg,224,224,1.0,RGB,R G B,False,False,Plumeria
3,/content/dataset/DSP305x_01-A_VN-dataset/train...,jpg,224,224,1.0,RGB,R G B,False,False,Plumeria
4,/content/dataset/DSP305x_01-A_VN-dataset/train...,jpg,224,224,1.0,RGB,R G B,False,False,Plumeria
...,...,...,...,...,...,...,...,...,...,...
20366,/content/dataset/DSP305x_01-A_VN-dataset/train...,png,224,224,1.0,RGB,R G B,False,False,Plum blossom
20367,/content/dataset/DSP305x_01-A_VN-dataset/train...,jpg,224,224,1.0,RGB,R G B,False,False,Plum blossom
20368,/content/dataset/DSP305x_01-A_VN-dataset/train...,jpg,224,224,1.0,RGB,R G B,False,False,Plum blossom
20369,/content/dataset/DSP305x_01-A_VN-dataset/train...,jpg,224,224,1.0,RGB,R G B,False,False,Plum blossom


## Tạo model

#### thiết lập mixed_precision


In [ ]:
# Enable mixed precision training
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy(policy="mixed_float16")
mixed_precision.global_policy()

<DTypePolicy "mixed_float16">

In [ ]:
train.shape, test.shape, val.shape

((20371, 10), (7426, 10), (8820, 10))

### Tạo tensorboard callbacks và train

In [ ]:
import tensorflow as tf
import datetime
from tensorflow.keras.applications import EfficientNetB2, InceptionV3, VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import os

In [ ]:
import datetime
import tensorflow as tf

def create_tensorboard_callback(dir_name, experiment_name):
    """
    Creates a TensorBoard callback instance to store log files.

    Stores log files with the filepath:
      "dir_name/experiment_name/current_datetime/"

    Args:
      dir_name: target directory to store TensorBoard log files
      experiment_name: name of experiment directory (e.g. efficientnet_model_1)
    """
    log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=log_dir
    )
    print(f"Saving TensorBoard log files to: {log_dir}")
    return tensorboard_callback

checkpoint_path = "model_checkpoints/.weights.h5"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path,
    monitor="val_accuracy",
    save_best_only=True,
    save_weights_only=True,
    verbose=0
)


### ImageDataGenerator : định dạng lại ảnh

In [ ]:
#
datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)


In [ ]:
# Create iterators for train, validation, and test data
train_generator = datagen.flow_from_dataframe(train, x_col='ImgPath', y_col='Class', target_size=(224, 224),batch_size=16)
val_generator = datagen.flow_from_dataframe(val, x_col='ImgPath', y_col='Class', target_size=(224, 224), batch_size = 16)
test_generator = datagen.flow_from_dataframe(test, x_col='ImgPath', y_col='Class', target_size=(224, 224), batch_size=16)


Found 20371 validated image filenames belonging to 10 classes.
Found 8820 validated image filenames belonging to 10 classes.
Found 7426 validated image filenames belonging to 10 classes.


In [ ]:
for data_batch, labels_batch in train_generator:
    print("Data batch shape:", data_batch.shape)
    print("Labels batch shape:", labels_batch.shape)
    break

Data batch shape: (16, 224, 224, 3)
Labels batch shape: (16, 10)


### Build model

In [ ]:
def build_model(base_model, num_classes):
    # Freeze the base model
    base_model.trainable = False

    # Define the input layer
    inputs = Input(shape=(224, 224, 3))

    # Pass the input through the base model
    x = base_model(inputs, training=False)

    # Global Average Pooling for downsampling the feature map
    x = GlobalAveragePooling2D()(x)

    # Flatten the output for the fully connected layers
    x = Flatten()(x)

    # Add a fully connected dense layer with ReLU activation
    x = Dense(128, activation="relu")(x)

    # Add a dropout layer for regularization
    x = Dropout(0.5)(x)

    # Final output layer with softmax activation for multi-class classification
    outputs = Dense(num_classes, activation="softmax")(x)

    # Create the model
    model = Model(inputs, outputs)
    return model


### Compile model  

In [ ]:
# Number of classes (e.g., 10 classes for 10 flower types)
num_classes = 10

# Initialize the pre-trained models
vgg16_model = build_model(VGG16(weights="imagenet", include_top=False), num_classes)
efficientnet_model = build_model(EfficientNetB2(weights="imagenet", include_top=False), num_classes)
inception_model = build_model(InceptionV3(weights="imagenet", include_top=False), num_classes)

# Compile the models with optimizer, loss, and metrics
vgg16_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
efficientnet_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
inception_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
31790344/31790344 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# vgg16_model.summary()

In [ ]:
# print("Training VGG16 model...")
# history_vgg = vgg16_model.fit(train_generator,
#                               epochs=10,
#                               validation_data=val_generator,
#                               callbacks=[model_checkpoint])

In [ ]:
# inception_model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ inception_v3 (Functional)            │ (None, 5, 5, 2048)          │      21,802,784 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_2           │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 2048)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 128)                 │         262,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 22,066,346 (84.18 MB)

 Trainable params: 263,562 (1.01 MB)

 Non-trainable params: 21,802,784 (83.17 MB)

In [ ]:
# print("Training InceptionV3 model...")

# history_inception = inception_model.fit(train_generator,
#                                   epochs=10,
#                                   validation_data=val_generator,
#                                   callbacks=[model_checkpoint])


Training InceptionV3 model...
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1274/1274 ━━━━━━━━━━━━━━━━━━━━ 404s 298ms/step - accuracy: 0.4229 - loss: 1.6577 - val_accuracy: 0.8244 - val_loss: 0.5915
Epoch 2/10
1274/1274 ━━━━━━━━━━━━━━━━━━━━ 355s 277ms/step - accuracy: 0.6635 - loss: 0.9345 - val_accuracy: 0.8601 - val_loss: 0.4554
Epoch 3/10
1274/1274 ━━━━━━━━━━━━━━━━━━━━ 365s 286ms/step - accuracy: 0.7072 - loss: 0.8131 - val_accuracy: 0.8902 - val_loss: 0.3903
Epoch 4/10
1274/1274 ━━━━━━━━━━━━━━━━━━━━ 372s 278ms/step - accuracy: 0.7328 - loss: 0.7281 - val_accuracy: 0.9032 - val_loss: 0.3095
Epoch 5/10
1274/1274 ━━━━━━━━━━━━━━━━━━━━ 381s 277ms/step - accuracy: 0.7589 - loss: 0.6657 - val_accuracy: 0.9137 - val_loss: 0.2887
Epoch 6/10
1274/1274 ━━━━━━━━━━━━━━━━━━━━ 355s 278ms/step - accuracy: 0.7561 - loss: 0.6731 - val_accuracy: 0.9137 - val_loss: 0.2859
Epoch 7/10
1274/1274 ━━━━━━━━━━━━━━━━━━━━ 353s 277ms/step - accuracy: 0.7634 - loss: 0.6478 - val_accuracy: 0.8961 - val_loss: 0.3121
Epoch 8/10
1274/1274 ━━━━━━━━━━━━━━━━━━━━ 382s 277ms/step - accuracy: 0.7

In [ ]:
efficientnet_model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ efficientnetb2 (Functional)          │ (None, 7, 7, 1408)          │       7,768,569 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_1           │ (None, 1408)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 1408)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │         180,352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,950,211 (30.33 MB)

 Trainable params: 181,642 (709.54 KB)

 Non-trainable params: 7,768,569 (29.63 MB)

In [ ]:
# Train the models
print("Training EfficientNetB0 model...")

history_eff = efficientnet_model.fit(train_generator, epochs=10,
                                     validation_data=val_generator,
                                     callbacks=[model_checkpoint])


Training EfficientNetB0 model...
Epoch 1/10
1146/1274 ━━━━━━━━━━━━━━━━━━━━ 27s 212ms/step - accuracy: 0.1002 - loss: 2.3125

## So sánh và chọn model tốt nhất cho bước tiếp theo

In [ ]:
%tensorboard --logdir logs
